In [ ]:
%pip install pandas
%pip install --upgrade torch 
%pip install torchvision
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117
import pandas as pd
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

#from PIL import Image (???)
#from torch.utils.data import Dataset (for custom datasets)

In [70]:
# Preparing the data
num_classes = 10
input_shape = (1, 28, 28)

# Load the data and split it between train and test sets
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.Compose([transforms.ToTensor()]))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [71]:
# Building the model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 4, 3, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(4 * 13 * 13, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 4 * 13 * 13)
        x = self.dropout(x)
        x = self.fc1(x)
        return F.log_softmax(x, dim=1)

model = Net()
print(model)

Net(
  (conv1): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=676, out_features=10, bias=True)
)


In [72]:
# Training the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item()))

In [73]:
# Evaluating the trained model
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

for epoch in range(1, 6):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302444
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.258052
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.678626
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.614861
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.587631
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.454680
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.832637
Train Epoch: 1 [22400/60000 (37%)]	Loss: 0.672267
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.447332
Train Epoch: 1 [28800/60000 (48%)]	Loss: 0.476280
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.516257
Train Epoch: 1 [35200/60000 (59%)]	Loss: 1.254317
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.218447
Train Epoch: 1 [41600/60000 (69%)]	Loss: 0.343588
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.426853
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.657658
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.229491
Train Epoch: 1 [54400/60000 (91%)]	Loss: 0.220436
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.449250

Test set: Average loss: 0.0087, Accuracy: 9260/10000 (92.6